### **DAY 9 (17/01/26) – SQL Analytics & Dashboards**

### Learn:

- SQL warehouses
- Complex analytical queries
- Dashboard creation
- Visualizations & filters

### 🛠️ Tasks:

1. Create SQL warehouse
2. Write analytical queries
3. Build dashboard: revenue trends, funnels, top products
4. Add filters & schedule refresh

# The queries of this challenge are written in SQL Editor -> Dashboards. And Re written here for reference purpose only, not for running.

![](/Volumes/ecommerce_catalog/default/img/Screenshot 2026-01-17 210344.png)

In [0]:
%sql
-- Sales revenue over time

USE CATALOG `workspace`;
USE SCHEMA `default`;

SELECT
  event_date,
  brand,
  SUM(price) AS total_revenue
FROM ecommerce_catalog.default.events_silver
WHERE event_type = 'purchase'
GROUP BY event_date, brand
ORDER BY event_date;


In [0]:
%sql

-- 7 Day Moving Average

WITH daily AS (
    SELECT
        event_date,
        SUM(price) AS revenue
    FROM ecommerce_catalog.default.events_silver
    WHERE event_type = 'purchase'
    GROUP BY event_date
)
SELECT
    event_date,
    revenue,
    ROUND(
        AVG(revenue) OVER (
            ORDER BY event_date
            ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
        ), 2
    ) AS revenue_ma7
FROM daily
ORDER BY event_date;


In [0]:
%sql
--Views - Purchases (conversion funnel)

SELECT
  stage,
  value
FROM (
  SELECT 'Views' AS stage, COUNT(*) AS value
  FROM ecommerce_catalog.default.events_silver
  WHERE event_type = 'view'

  UNION ALL

  SELECT 'Purchases' AS stage, COUNT(*) AS value
  FROM ecommerce_catalog.default.events_silver
  WHERE event_type = 'purchase'
);


In [0]:
%sql
-- Customer tiers (VIP / Loyal / Regular)

WITH customer_stats AS (
    SELECT
        user_id,
        COUNT(*) AS purchase_count,
        SUM(price) AS total_spent
    FROM ecommerce_catalog.default.events_silver
    WHERE event_type = 'purchase'
    GROUP BY user_id
)
SELECT
    CASE
        WHEN purchase_count >= 10 THEN 'VIP'
        WHEN purchase_count >= 5 THEN 'Loyal'
        ELSE 'Regular'
    END AS customer_tier,
    COUNT(*) AS customers,
    ROUND(AVG(total_spent), 2) AS avg_lifetime_value
FROM customer_stats
GROUP BY customer_tier
ORDER BY avg_lifetime_value DESC;
